# Dynamic Imputation RNN
### Prediction Imputation

- For this model we do not mask the training set's missing instances. We impute them.
- We do, however, perform masking for the teting data evaluation.

# Updates
- Remove all test operations
- mask_list.append((train, test)) went to mask_list.append(train)

# Libraries

In [4]:
from __future__ import print_function
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import tensorflow as tf
import sklearn as sk
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold

from keras import callbacks
import keras.layers as L
import keras.models as M
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error as mse
import os, shutil

from keras.regularizers import l1_l2

### Functions

In [5]:
def select_columns(col_list, n_months):
    """
    Takes in a list of column names and number of visits starting at 0.
    Returns column list time-stepped and dovetailed.
    """ 
    return dovetail_names(*[time_step_names(i, n_months) for i in col_list])
        
def time_step_names(name, n_months):

    return [(name + '_%d' % (j+1)) for j in range(-1,n_months*6, 6)]

def dovetail_names(*kwargs):
    """
    Dovetails column names across time slices cccording to preset order.
    """
    zipped = zip(*kwargs)
    l = []
   
    for i in zipped:
        
        for j in i:
            
            l.append(j)
            
    return l

def stretch_input(Xtr, time_steps, pot, n_features=115,):
    """
    Xtr_fill is empty 3D numpy array where we extend length of patient observation times t
    pot stands for Patient Observation Time. We only need to do this for our X input
    """
    Xtr_fill = np.zeros(shape=[Xtr.shape[0],time_steps,n_features*pot] , dtype = object) 

    for subject in range(Xtr.shape[0]):
    
        for i in range(time_steps):

            temp = np.concatenate([Xtr[subject][i],Xtr[subject][i+1]]) # changed for pot = 3
            Xtr_fill[subject][i] = temp
            
    return Xtr_fill

def reshape_data(X, y, n_time_steps, n_features = 115):  
    
    X_reshaped = X.values.reshape(-1, n_time_steps, n_features)
    y_reshaped = y.values.reshape(-1, n_time_steps, 1)
    print("X reshaped is " + str(X_reshaped.shape))
    print("y reshaped is " + str(y_reshaped.shape))

    y = y_reshaped.astype(float)
    X = X_reshaped.astype(float)
    
    print(X.shape, y.shape)
    return X, y

def provide_data(X,y,roll,n_features=115):
 
    X = X.iloc[:,:n_features*roll]
    y = y.iloc[:,:roll]

    y_full = y.dropna()

    mask = X.index.isin(y_full.index.tolist())

    X_full = X[mask]

    y_nan = y[~mask]
    X_nan = X[~mask]
    
    print('NaN')
    X_nan, y_nan = reshape_data(X_nan, y_nan, roll)
    print('Full')
    X_full, y_full = reshape_data(X_full, y_full, roll)
    
    return X_full, X_nan, y_full, y_nan, mask

def provide_all_data(X,y,roll):
 
    X = X.iloc[:,:n_features*roll]
    y = y.iloc[:,:roll]

    X_all, y_all = reshape_data(X, y, roll)
    
    return X_all, y_all
    
def prepare_for_mask(X,y, mask_value = -99):
    
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            if np.isnan(y[i][j][0]) or (y[i][j] == mask_value):

                X[i][j] = mask_value
                y[i][j] = mask_value
                
            if (mask_value in X[i][j]) or np.isnan(X[i][j][114]):
                
                X[i][j] = mask_value
                y[i][j] = mask_value
            
    return X,y

def round_off_EDSS(number):
    """Round a number to the closest half integer.
    >>> round_of_rating(1.3)
    1.5
    >>> round_of_rating(2.6)
    2.5
    >>> round_of_rating(3.0)
    3.0
    >>> round_of_rating(4.1)
    4.0"""
    return np.round(number * 2) / 2
   
def def_train_name(f_ix):
    
    X_train_name = "data_folds/X_train_f" + str(f_ix + 1) + ".csv"
    y_train_name = "data_folds/y_train_f" + str(f_ix + 1) + ".csv"
    X_test_name = "data_folds/X_test_f" + str(f_ix + 1) + ".csv"
    y_test_name = "data_folds/y_test_f" + str(f_ix + 1) + ".csv"
    
    return X_train_name, X_test_name, y_train_name, y_test_name

# Keras Callbacks

In [6]:
weight_file_path = "weights/my_model_weights_6m.h5"
final_file_path = "final_weights/my_model_weights_6m.h5"
best_file_path = "best/best_weights_6m.hdf5"

# reduce learning rate on plateau
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                               patience=5, min_lr=0.001)

# stop training if there isn't a significant improvement in the course of 5 epochs
early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, 
                              patience=5, verbose=0, mode='auto', 
                              baseline=None, restore_best_weights=True)

# model check point
model_checkpoint = ModelCheckpoint(best_file_path, monitor='val_loss', 
                                   verbose=1, 
                                   save_best_only=True, 
                                   mode='min')

callbacks_list = [reduce_lr, early_stopping]
callbacks_list_final = [reduce_lr, early_stopping, model_checkpoint]

In [7]:
def rnn_model(n_time_steps, n_inputs):
    
    m = M.Sequential()
    m.add(L.Masking(mask_value=-99, input_shape=(n_time_steps, n_inputs)))
    m.add(L.GRU(128, return_sequences=True))
    m.add(L.TimeDistributed(L.Dense(1, activation='relu')))
    m.compile(optimizer = 'nadam', loss = 'mean_absolute_error')

    return m

# Part 1 

- Unroll RNN one time step at a time
- Transfer weights between rolls

In [8]:
def generate_initial_predictions(n_time_steps, X_train, y_train, X_test, y_test, n_inputs=115):
    
    threshold = len(y_train.columns)
    print("TRAIN")
    X_full_train, X_nan_train, y_full_train, y_nan_train, mask_train = provide_data(X_train, y_train, n_time_steps)
    print("TEST")
    X_full_test, X_nan_test, y_full_test, y_nan_test, mask_test = provide_data(X_test, y_test, n_time_steps)
    
    K.clear_session()
    
    m = rnn_model(n_time_steps, n_inputs)

    if n_time_steps > 1:
        
        # load weights from previous model to establish continuity 
        m.load_weights(weight_file_path)
    
    history = m.fit(X_full_train, y_full_train, 
                    validation_split=0.2,
                    batch_size = X_full_train.shape[0], # batch size == size of the training data 
                    epochs=1, 
                    shuffle=True,
                    callbacks = callbacks_list)
        
    m.save_weights(weight_file_path) # save weights 
    
    # Predict for unknown values of training and testing data
    # Using train data with known targets, I train a model to predict for missing values
    # In both training and testing data 
    # But for the testing data, I only impute the X feature space 
    
    m_pred_train = round_off_EDSS(
        pd.Series(np.array([x[0] for x in m.predict(X_nan_train)]).flatten(),
                             index = y_train[~mask_train].index))
    
    # redundant but also helps me known which values to update for test data feature space
    m_pred_test = round_off_EDSS(pd.Series(np.array([x[0] for x in m.predict(X_nan_test)]).flatten(),
                           index = y_test[~mask_test].index))
    
    # In pandas the best way to update a dataframe or series
    # is with another dataframe or series with the desired index
    y_train.iloc[:,n_time_steps-1].update(m_pred_train)
    
    if (n_time_steps < threshold): # when y contains an EDSS column that X does not 
        
        X_train.iloc[:,(n_features*(n_time_steps+1))-1].update(m_pred_train)
        # redundant but also helps me known which values to update for test data feature space
        X_test.iloc[:,(n_features*(n_time_steps+1))-1].update(m_pred_test)
        
    if (n_time_steps == threshold):
        m.save_weights(final_file_path)
        
    X_train_all, y_train_all = provide_all_data(X_train, y_test, n_time_steps)
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    masked_X_train_all, masked_y_train_all = prepare_for_mask(X_train_all, y_train_all)
    masked_X_test_all, masked_y_test_all = prepare_for_mask(X_test_all, y_test_all)
    
    # predict for all values 
    # use the masking list to known which of these should be used as imputation values
    all_train_pred = round_off_EDSS(m.predict(masked_X_train_all))
    all_test_pred = round_off_EDSS(m.predict(masked_X_test_all))

    return X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test 

def update_previous_predictions(n_time_steps, all_train_pred, all_test_pred):
    
    m_pred_train = np.array(all_train_pred[:,n_time_steps-1,:]).flatten()
    m_pred_test = np.array(all_test_pred[:,n_time_steps-1,:]).flatten()
    
    return m_pred_train, m_pred_test

def impute_previous_predictions(n_time_steps, X_train, X_test,y_train,
                                m_pred_train, m_pred_test, mask_list, n_inputs=115):
    
    mask_train = mask_list[n_time_steps-1][0]
    mask_test = mask_list[n_time_steps-1][1]
    
    m_pred_train = pd.Series(m_pred_train[~mask_train], index = y_train[~mask_train].index)
    m_pred_test = pd.Series(m_pred_test[~mask_test], index = y_test[~mask_test].index)
    
    y_train.iloc[:, n_time_steps-1].update(m_pred_train)
    
    if n_time_steps < len(y_train.columns): # 36
        
        X_train.iloc[:,(n_features*(n_time_steps+1))-1].update(m_pred_train)
        X_test.iloc[:,(n_features*(n_time_steps+1))-1].update(m_pred_test)
    
    return X_train, X_test, y_train # y test does not get imputed

# Part 2

- Iterate predictive imputation for all time steps 
- Update all imputated values for each iteration

In [9]:
def exhaustive_imputation(X_train, y_train, X_test, y_test, n_features=115, pot = 1):
    
    """Running the model after we've impute for the whole feature space"""
    
    n_inputs = n_features*pot
    n_time_steps = len(y_train.columns)
    
    X_train_all, y_train_all = provide_all_data(X_train, y_train, n_time_steps)
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    masked_X_test_all, masked_y_test_all = prepare_for_mask(X_test_all, y_test_all)
    
    K.clear_session()

    m = rnn_model(n_time_steps, n_inputs)
    
    # load weights from previous model to establish continuity 
    m.load_weights(final_file_path)
    
    history = m.fit(X_train_all, y_train_all,
                    validation_split = 0.2, 
                    batch_size = X_train_all.shape[0], # of training instances == batch_size 
                    epochs=1,
                    shuffle=True,
                   callbacks = callbacks_list_final) # should save best weights 
    
    m.save_weights(final_file_path)
    
    all_train_pred = round_off_EDSS(m.predict(X_train_all))
    all_test_pred = round_off_EDSS(m.predict(masked_X_test_all))
    
    val_loss = history.__dict__['history']['val_loss']
    
    return all_train_pred, all_test_pred, val_loss, m

def iterate_exhaustive_imputation(n_epochs, X_train, y_train, X_test, y_test):
    """
    We divide our RNN into different 1 epoch models 
    in order to iteratively update the imputations
    """
    val_loss_list = []
    
    for i in range(n_epochs):
        
        print('EPOCH: ', str(i))
        all_train_pred, all_test_pred, val_loss, model = exhaustive_imputation(X_train, y_train, 
                                                                               X_test, y_test)

        val_loss_list.append(val_loss)
    
        for i in range(1, all_train_pred.shape[1]): # -1 

                    m_pred_train, m_pred_test = update_previous_predictions(i, all_train_pred, 
                                                                            all_test_pred)    
                    
                    X_train, X_test, y_train = impute_previous_predictions(i, X_train, X_test,
                                                                           y_train, m_pred_train, 
                                                                           m_pred_test, mask_list)
    
    return np.min(val_loss_list), model

# Part 3

- Generate final imputation and return imputed datasets

In [10]:
def best_weights_imputation(X_train, y_train, X_test, y_test, n_features=115, pot = 1):
    
    """
    Returns EDSS imputations using best model weights
    Running the model after we've impute for the whole feature space"""
    n_inputs = n_features*pot
    n_time_steps = len(y_train.columns)
    
    X_train_all, y_train_all = provide_all_data(X_train, y_train, n_time_steps)
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    # we do want to mask the instances of the testing data where there are missing targets
    # so that it does not affect our val_loss
    masked_X_test_all, masked_y_test_all = prepare_for_mask(X_test_all, y_test_all)
    
    K.clear_session()

    m = rnn_model(n_time_steps, n_inputs)
    # load weights from best weights model to generate final imputation values 
    m.load_weights(best_file_path)
    
    all_train_pred = round_off_EDSS(m.predict(X_train_all))
    all_test_pred = round_off_EDSS(m.predict(masked_X_test_all))
    
    return all_train_pred, all_test_pred

def return_imputed_data(X_train, X_test, y_train, y_test):
    
    """
    Part 2
    Returns final data sets imputed with best model weights
    [Feature EDSS (train and test); target EDSS (train)]
    """
    
    val_loss_list = []
    
    all_train_pred, all_test_pred = best_weights_imputation(X_train, y_train, X_test, y_test)
    
    for i in range(1, all_train_pred.shape[1]): # -1 

                m_pred_train, m_pred_test = update_previous_predictions(i, all_train_pred, all_test_pred)    
                    
                X_train, X_test, y_train = impute_previous_predictions(i, X_train, X_test,y_train, m_pred_train, m_pred_test, mask_list)
    
    n_time_steps = len(y_test.columns)
    
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    return X_train, X_test, y_train, y_test

# Part 4

- Train new models on imputed datasets

In [11]:
def optimal_test_model(X_train, y_train, n_features=115, pot = 1):
    
    """STEP 3, PART A
    Determine optimal model to evalute test data
    now that the data sets are already imputed"""
    # Delete Previous Model Checkpoint, if any
    if os.path.exists('optimal_weights_6m/'):
        shutil.rmtree('optimal_weights_6m/')
    os.makedirs('optimal_weights_6m/')
        
    optimal_file_path = "optimal_weights_6m/weights_6m.hdf5"
    
    model_checkpoint = ModelCheckpoint(optimal_file_path, 
                                       monitor='val_loss', 
                                       verbose=1, 
                                       save_best_only=True, mode='min')

    callbacks_list = [reduce_lr, early_stopping, model_checkpoint]

    """Running the model after we've imputed for the whole feature space"""
    
    n_inputs = n_features*pot
    n_time_steps = len(y_train.columns)
    
    X_train_all, y_train_all = provide_all_data(X_train, y_train, n_time_steps)
    
    K.clear_session()
    
    m = rnn_model(n_time_steps, n_inputs)
    
    m.fit(X_train_all, y_train_all,
                    validation_split = 0.2,
                    batch_size = 32, 
                    epochs=100,
                    shuffle=True,
                   callbacks = callbacks_list)
    
    K.clear_session()
    
    m = rnn_model(n_time_steps, n_inputs)
    # load weights from previous model to establish continuity 
    m.load_weights(optimal_file_path)
    
    return m

def evaluate_model(X_train, X_test, y_train, y_test): 
    """
    STEP 3 PART B
    Evaluate model with held out test set
    """
    
    m = optimal_test_model(X_train, y_train)
    
    n_time_steps = len(y_train.columns)
    
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    masked_X_test_all, masked_y_test_all = prepare_for_mask(X_test_all, y_test_all)
    
    res = m.evaluate(masked_X_test_all, masked_y_test_all)
    print(res)
    
    return res

# Import Data

In [12]:
X = pd.read_csv('../../../data/pre_imputation_data/X_6_months|6_months.csv', index_col = 0)
y = pd.read_csv('../../../data/pre_imputation_data/y_6_months|6_months.csv', index_col = 0)

#n_time_steps = len(y.columns)
n_features = X.columns.tolist().index("EDSS_0")+1
pot = 1
n_inputs = n_features * pot
n_units = 128
skf = KFold(n_splits=5, shuffle=True, random_state=42)

print("The input length of the training data will be", pot, "time slices, separated by 6 month intervals")
print(n_features, "featues comprise one time slice")

# Drop all missing values from EDSS_0 which is our first X feature predictor
mask = X.index.isin(X['EDSS_0'].dropna().index.tolist())

X_og, y_og = X[mask], y[mask]

The input length of the training data will be 1 time slices, separated by 6 month intervals
115 featues comprise one time slice


# Divide Data into Training and Testing

In [13]:
for f_ix in range(5):

    train_ix = []
    test_ix = []
    for train, test in skf.split(X_og.index):
        train_ix.append(train)
        test_ix.append(test)

    X_train_og, y_train_og = X_og.iloc[train_ix[f_ix], :].copy(), y_og.iloc[train_ix[f_ix], :].copy()
    X_test_og, y_test_og = X_og.iloc[test_ix[f_ix], :].copy(), y_og.iloc[test_ix[f_ix], :].copy()

    # Run algorithm

    mask_list = []

    X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test = generate_initial_predictions(1, X_train_og, 
                                                                                        y_train_og, 
                                                                                        X_test_og, 
                                                                                        y_test_og)

    mask_list.append((mask_train, mask_test)) # these tell me what values are missing in each column 

    # Sanity Check

    print("There are missing values for EDSS in the X feature space:", pd.isna(X_train['EDSS_6']).any())
    print("There are missing values for EDSS in the y target space:", pd.isna(y_train['EDSS_6']).any())

    for r in range(2, len(y_train.columns)+1):

        X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test = generate_initial_predictions(r, X_train, y_train, X_test, y_test)

        mask_list.append((mask_train, mask_test))

        if (all_train_pred.shape[1] > 1):

            # update all previous predictions
            for col in range(1,all_train_pred.shape[1]):
                # for each column of the training and testing EDSS
                m_pred_train, m_pred_test = update_previous_predictions(col, all_train_pred, all_test_pred)    
                X_train, X_test, y_train = impute_previous_predictions(col, X_train, X_test, y_train,m_pred_train, m_pred_test, mask_list)


    # X_train should go up to 210

    save_temp = (X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test)
    save_list_temp = mask_list.copy()

    X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test = save_temp 
    mask_list = save_list_temp

    # Run final model for training with whole feature space and training targets

    best, model = iterate_exhaustive_imputation(100, X_train, y_train, X_test, y_test) # use original y test

    best

    # Generate optimal imputation 

    X_train, X_test, y_train, y_test =  return_imputed_data(X_train, X_test, y_train, y_test) # use original y test

    # Use original unimputed test data and save data

    X_test_no_imputation, y_test_no_imputation = X_og.iloc[test_ix[f_ix], :].copy(), y_og.iloc[test_ix[f_ix], :].copy()

    # save data folds
    a,b,c,d = def_train_name(f_ix)

    X_train.to_csv(a)
    X_test_no_imputation.to_csv(b)
    y_train.to_csv(c)
    y_test_no_imputation.to_csv(d)

TRAIN
NaN
X reshaped is (62, 1, 115)
y reshaped is (62, 1, 1)
(62, 1, 115) (62, 1, 1)
Full
X reshaped is (330, 1, 115)
y reshaped is (330, 1, 1)
(330, 1, 115) (330, 1, 1)
TEST
NaN
X reshaped is (15, 1, 115)
y reshaped is (15, 1, 1)
(15, 1, 115) (15, 1, 1)
Full
X reshaped is (84, 1, 115)
y reshaped is (84, 1, 1)
(84, 1, 115) (84, 1, 1)
Train on 264 samples, validate on 66 samples
Epoch 1/1
264/264 [==============================] - 2s 6ms/step - loss: 1.8475 - val_loss: 1.4103
X reshaped is (392, 1, 115)
y reshaped is (99, 1, 1)
(392, 1, 115) (99, 1, 1)
X reshaped is (99, 1, 115)
y reshaped is (99, 1, 1)
(99, 1, 115) (99, 1, 1)
There are missing values for EDSS in the X feature space: False
There are missing values for EDSS in the y target space: False
TRAIN
NaN
X reshaped is (33, 2, 115)
y reshaped is (33, 2, 1)
(33, 2, 115) (33, 2, 1)
Full
X reshaped is (359, 2, 115)
y reshaped is (359, 2, 1)
(359, 2, 115) (359, 2, 1)
TEST
NaN
X reshaped is (19, 2, 115)
y reshaped is (19, 2, 1)
(19, 2

212/212 [==============================] - 2s 7ms/step - loss: 0.6423 - val_loss: 1.0559
X reshaped is (392, 13, 115)
y reshaped is (99, 13, 1)
(392, 13, 115) (99, 13, 1)
X reshaped is (99, 13, 115)
y reshaped is (99, 13, 1)
(99, 13, 115) (99, 13, 1)
TRAIN
NaN
X reshaped is (123, 14, 115)
y reshaped is (123, 14, 1)
(123, 14, 115) (123, 14, 1)
Full
X reshaped is (269, 14, 115)
y reshaped is (269, 14, 1)
(269, 14, 115) (269, 14, 1)
TEST
NaN
X reshaped is (78, 14, 115)
y reshaped is (78, 14, 1)
(78, 14, 115) (78, 14, 1)
Full
X reshaped is (21, 14, 115)
y reshaped is (21, 14, 1)
(21, 14, 115) (21, 14, 1)
Train on 215 samples, validate on 54 samples
Epoch 1/1
215/215 [==============================] - 2s 7ms/step - loss: 0.8834 - val_loss: 1.0317
X reshaped is (392, 14, 115)
y reshaped is (99, 14, 1)
(392, 14, 115) (99, 14, 1)
X reshaped is (99, 14, 115)
y reshaped is (99, 14, 1)
(99, 14, 115) (99, 14, 1)
TRAIN
NaN
X reshaped is (150, 15, 115)
y reshaped is (150, 15, 1)
(150, 15, 115) (150,

TRAIN
NaN
X reshaped is (301, 26, 115)
y reshaped is (301, 26, 1)
(301, 26, 115) (301, 26, 1)
Full
X reshaped is (91, 26, 115)
y reshaped is (91, 26, 1)
(91, 26, 115) (91, 26, 1)
TEST
NaN
X reshaped is (97, 26, 115)
y reshaped is (97, 26, 1)
(97, 26, 115) (97, 26, 1)
Full
X reshaped is (2, 26, 115)
y reshaped is (2, 26, 1)
(2, 26, 115) (2, 26, 1)
Train on 72 samples, validate on 19 samples
Epoch 1/1
72/72 [==============================] - 2s 23ms/step - loss: 0.6602 - val_loss: 0.7036
X reshaped is (392, 26, 115)
y reshaped is (99, 26, 1)
(392, 26, 115) (99, 26, 1)
X reshaped is (99, 26, 115)
y reshaped is (99, 26, 1)
(99, 26, 115) (99, 26, 1)
TRAIN
NaN
X reshaped is (296, 27, 115)
y reshaped is (296, 27, 1)
(296, 27, 115) (296, 27, 1)
Full
X reshaped is (96, 27, 115)
y reshaped is (96, 27, 1)
(96, 27, 115) (96, 27, 1)
TEST
NaN
X reshaped is (97, 27, 115)
y reshaped is (97, 27, 1)
(97, 27, 115) (97, 27, 1)
Full
X reshaped is (2, 27, 115)
y reshaped is (2, 27, 1)
(2, 27, 115) (2, 27, 1

313/313 [==============================] - 2s 6ms/step - loss: 0.4835 - val_loss: 1.0790

Epoch 00001: val_loss did not improve from 0.64514
EPOCH:  3
X reshaped is (392, 36, 115)
y reshaped is (392, 36, 1)
(392, 36, 115) (392, 36, 1)
X reshaped is (99, 36, 115)
y reshaped is (99, 36, 1)
(99, 36, 115) (99, 36, 1)
Train on 313 samples, validate on 79 samples
Epoch 1/1
313/313 [==============================] - 2s 6ms/step - loss: 0.8163 - val_loss: 0.8337

Epoch 00001: val_loss did not improve from 0.64514
EPOCH:  4
X reshaped is (392, 36, 115)
y reshaped is (392, 36, 1)
(392, 36, 115) (392, 36, 1)
X reshaped is (99, 36, 115)
y reshaped is (99, 36, 1)
(99, 36, 115) (99, 36, 1)
Train on 313 samples, validate on 79 samples
Epoch 1/1
313/313 [==============================] - 2s 6ms/step - loss: 0.5204 - val_loss: 1.0083

Epoch 00001: val_loss did not improve from 0.64514
EPOCH:  5
X reshaped is (392, 36, 115)
y reshaped is (392, 36, 1)
(392, 36, 115) (392, 36, 1)
X reshaped is (99, 36, 11


Epoch 00001: val_loss did not improve from 0.64514
EPOCH:  25
X reshaped is (392, 36, 115)
y reshaped is (392, 36, 1)
(392, 36, 115) (392, 36, 1)
X reshaped is (99, 36, 115)
y reshaped is (99, 36, 1)
(99, 36, 115) (99, 36, 1)
Train on 313 samples, validate on 79 samples
Epoch 1/1
313/313 [==============================] - 2s 6ms/step - loss: 0.4774 - val_loss: 0.7364

Epoch 00001: val_loss did not improve from 0.64514
EPOCH:  26
X reshaped is (392, 36, 115)
y reshaped is (392, 36, 1)
(392, 36, 115) (392, 36, 1)
X reshaped is (99, 36, 115)
y reshaped is (99, 36, 1)
(99, 36, 115) (99, 36, 1)
Train on 313 samples, validate on 79 samples
Epoch 1/1
313/313 [==============================] - 2s 6ms/step - loss: 0.5441 - val_loss: 0.9584

Epoch 00001: val_loss did not improve from 0.64514
EPOCH:  27
X reshaped is (392, 36, 115)
y reshaped is (392, 36, 1)
(392, 36, 115) (392, 36, 1)
X reshaped is (99, 36, 115)
y reshaped is (99, 36, 1)
(99, 36, 115) (99, 36, 1)
Train on 313 samples, validate 

...

Train on 28 samples, validate on 8 samples
Epoch 1/1
28/28 [==============================] - 2s 58ms/step - loss: 0.6813 - val_loss: 0.8977
X reshaped is (393, 32, 115)
y reshaped is (98, 32, 1)
(393, 32, 115) (98, 32, 1)
X reshaped is (98, 32, 115)
y reshaped is (98, 32, 1)
(98, 32, 115) (98, 32, 1)
TRAIN
NaN
X reshaped is (370, 33, 115)
y reshaped is (370, 33, 1)
(370, 33, 115) (370, 33, 1)
Full
X reshaped is (23, 33, 115)
y reshaped is (23, 33, 1)
(23, 33, 115) (23, 33, 1)
TEST
NaN
X reshaped is (98, 33, 115)
y reshaped is (98, 33, 1)
(98, 33, 115) (98, 33, 1)
Full
X reshaped is (0, 33, 115)
y reshaped is (0, 33, 1)
(0, 33, 115) (0, 33, 1)
Train on 18 samples, validate on 5 samples
Epoch 1/1
18/18 [==============================] - 2s 89ms/step - loss: 0.7210 - val_loss: 0.7372
X reshaped is (393, 33, 115)
y reshaped is (98, 33, 1)
(393, 33, 115) (98, 33, 1)
X reshaped is (98, 33, 115)
y reshaped is (98, 33, 1)
(98, 33, 115) (98, 33, 1)
TRAIN
NaN
X reshaped is (382, 34, 115)
y resh

Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4317 - val_loss: 1.0806

Epoch 00001: val_loss did not improve from 0.43967
EPOCH:  15
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.7369 - val_loss: 0.6293

Epoch 00001: val_loss did not improve from 0.43967
EPOCH:  16
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4102 - val_loss: 0.9436

Epoch 00001: val_loss did not improve from 0.43967
EPOCH:  17
X reshaped is (393, 36, 115)
y reshaped is (393, 36

314/314 [==============================] - 2s 6ms/step - loss: 0.4407 - val_loss: 1.0662

Epoch 00001: val_loss did not improve from 0.43967
EPOCH:  37
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.7514 - val_loss: 0.6660

Epoch 00001: val_loss did not improve from 0.43967
EPOCH:  38
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4120 - val_loss: 0.7971

Epoch 00001: val_loss did not improve from 0.43967
EPOCH:  39
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36,


Epoch 00001: val_loss did not improve from 0.43967
EPOCH:  59
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.7153 - val_loss: 0.7682

Epoch 00001: val_loss did not improve from 0.43967
EPOCH:  60
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4219 - val_loss: 1.0265

Epoch 00001: val_loss did not improve from 0.43967
EPOCH:  61
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate 

EPOCH:  81
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.6090 - val_loss: 0.7693

Epoch 00001: val_loss did not improve from 0.43967
EPOCH:  82
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4412 - val_loss: 0.7035

Epoch 00001: val_loss did not improve from 0.43967
EPOCH:  83
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [===================

# Evaluate models 

result1 = evaluate_model(X_train,  X_test_no_imputation, y_train, y_test_no_imputation) # use original y test

less = 1
result2 = evaluate_model(X_train.iloc[:,:-(less*115)],
               X_test_no_imputation.iloc[:,:-(less*115)], 
               y_train.iloc[:,less:], 
               y_test_no_imputation.iloc[:,less:]) # use original y test

less = 2
result3 = evaluate_model(X_train.iloc[:,:-(less*115)],
               X_test_no_imputation.iloc[:,:-(less*115)], 
               y_train.iloc[:,less:], 
               y_test_no_imputation.iloc[:,less:]) # use original y test

less = 3
result4 = evaluate_model(X_train.iloc[:,:-(less*115)],
               X_test_no_imputation.iloc[:,:-(less*115)], 
               y_train.iloc[:,less:], 
               y_test_no_imputation.iloc[:,less:]) # use original y test

less = 4
result5 = evaluate_model(X_train.iloc[:,:-(less*115)],
               X_test_no_imputation.iloc[:,:-(less*115)], 
               y_train.iloc[:,less:], 
               y_test_no_imputation.iloc[:,less:]) # use original y test

less = 5
result6 = evaluate_model(X_train.iloc[:,:-(less*115)],
               X_test_no_imputation.iloc[:,:-(less*115)], 
               y_train.iloc[:,less:], 
               y_test_no_imputation.iloc[:,less:]) # use original y test

less = 6
result7 = evaluate_model(X_train.iloc[:,:-(less*115)],
               X_test_no_imputation.iloc[:,:-(less*115)], 
               y_train.iloc[:,less:], 
               y_test_no_imputation.iloc[:,less:]) # use original y test

less = 7
result8 = evaluate_model(X_train.iloc[:,:-(less*115)],
               X_test_no_imputation.iloc[:,:-(less*115)], 
               y_train.iloc[:,less:], 
               y_test_no_imputation.iloc[:,less:]) # use original y test

less = 8
result9 = evaluate_model(X_train.iloc[:,:-(less*115)],
               X_test_no_imputation.iloc[:,:-(less*115)], 
               y_train.iloc[:,less:], 
               y_test_no_imputation.iloc[:,less:]) # use original y test

less = 9
result10 = evaluate_model(X_train.iloc[:,:-(less*115)],
               X_test_no_imputation.iloc[:,:-(less*115)], 
               y_train.iloc[:,less:], 
               y_test_no_imputation.iloc[:,less:]) # use original y test

fold = pd.DataFrame({'F1':[result1, result2, result3, result4, result5, result6, result7, result8, result9, result10]})

#fold.to_csv('data_folds/F1.csv')

fold